In [2]:
import pandas as pd
import requests
import io
import os
import csv
import json
import ast
import pprint
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/54.0.2840.90 '
                  'Safari/537.36'
}

### Exploring ONS API

In [3]:
api_url = 'https://api.beta.ons.gov.uk/v1/'

In [4]:
offset = 0
limit = 500

response = requests.get(f'{api_url}datasets', params={"limit": limit, "offset": offset})
response.raise_for_status()  # raises exception when not a 2xx response
available_datasets_json = response.json()

datasets = []
for dictionary in available_datasets_json['items']:
    datasets.append([dictionary['title'], dictionary['id']])

datasets

[['Quarterly personal well-being estimates', 'wellbeing-quarterly'],
 ['Personal well-being estimates by local authority',
  'wellbeing-local-authority'],
 ['Deaths registered weekly in England and Wales by region',
  'weekly-deaths-region'],
 ['Death registrations and occurrences by local authority and place of death',
  'weekly-deaths-local-authority'],
 ['Death registrations and occurrences by health board and place of death',
  'weekly-deaths-health-board'],
 ['Deaths registered weekly in England and Wales by age and sex',
  'weekly-deaths-age-sex'],
 ['UK spending on credit and debit cards', 'uk-spending-on-cards'],
 ['UK Business: Activity, Size and Location',
  'uk-business-by-enterprises-and-local-units'],
 ['Traffic Camera Activity', 'traffic-camera-activity'],
 ['Trade in goods: country by commodity', 'trade'],
 ['Effects of Taxes and Benefits on Household Income',
  'tax-benefits-statistics'],
 ['Suicide registrations in England and Wales by local authority',
  'suicides-in-

In [5]:
ons_ids = ['wellbeing-quarterly', 
       'wellbeing-local-authority',
       'uk-spending-on-cards',
       'tax-benefits-statistics',
       'regional-gdp-by-quarter',
       'labour-market',
       'gdp-to-four-decimal-places',
       'gdp-by-local-authority',
       'cpih01',
       'ashe-tables-20',
      ]

# get BoE data for interest rates and mortgage approval rates

#### Function to retrieve ONS dataset using its id

In [8]:
def get_ONS_dataset(id: str, api_url='https://api.beta.ons.gov.uk/v1/'):
    response = requests.get(f'{api_url}datasets/{id}')
    dataset_json = response.json()
    dataset_latest = dataset_json['links']['latest_version']['href']
    dataset_meta_json = requests.get(f'{dataset_latest}/metadata').json()
    dataset_latest_csv = dataset_meta_json['downloads']['csv']['href']
    return pd.read_csv(dataset_latest_csv, storage_options = {'User-Agent': 'Mozilla/5.0'})

#### 1. Wellbeing quarterly - ONS

In [9]:
wellbeing_q_df = get_ONS_dataset(ons_ids[0])
wellbeing_q_df.head()

,v4_2,LCL,UCL,yyyy-qq,Time,uk-only,Geography,measure-of-wellbeing,MeasureOfWellbeing,wellbeing-estimate,Estimate,seasonal-adjustment,SeasonalAdjustment
0,7.5,NaN,NaN,2023-q1,2023 Q1,K02000001,United Kingdom,life-satisfaction,Life satisfaction,average-mean,Average (mean),seasonal-adjustment,Seasonally adjusted
1,16.3,15.4,17.2,2023-q1,2023 Q1,K02000001,United Kingdom,life-satisfaction,Life satisfaction,fair,Fair,non-seasonal-adjustment,Non-seasonally adjusted
2,54.1,53.0,55.3,2023-q1,2023 Q1,K02000001,United Kingdom,life-satisfaction,Life satisfaction,good,Good,non-seasonal-adjustment,Non-seasonally adjusted
3,23.7,22.8,24.7,2023-q1,2023 Q1,K02000001,United Kingdom,life-satisfaction,Life satisfaction,very-good,Very good,non-seasonal-adjustment,Non-seasonally adjusted
4,17.3,NaN,NaN,2023-q1,2023 Q1,K02000001,United Kingdom,happiness,Happiness,fair,Fair,seasonal-adjustment,Seasonally adjusted


#### 2. Wellbeing local authority - ONS

In [10]:
wellbeing_la_df = get_ONS_dataset(ons_ids[1])
wellbeing_la_df.head()

,v4_3,Data marking,Lower limit,Upper limit,yyyy-yy,Time,administrative-geography,Geography,measure-of-wellbeing,MeasureOfWellbeing,wellbeing-estimate,Estimate
0,NaN,[x],[x],[x],2015-16,2015-16,E13000002,Outer London,life-satisfaction,Life satisfaction,fair,Fair
1,NaN,[x],[x],[x],2016-17,2016-17,E13000002,Outer London,life-satisfaction,Life satisfaction,good,Good
2,NaN,[x],[x],[x],2018-19,2018-19,E13000002,Outer London,life-satisfaction,Life satisfaction,fair,Fair
3,NaN,[x],[x],[x],2018-19,2018-19,E13000002,Outer London,life-satisfaction,Life satisfaction,very-good,Very good
4,NaN,[x],[x],[x],2014-15,2014-15,E13000002,Outer London,worthwhile,Worthwhile,fair,Fair


#### 3. UK spending on debit and credit cards

In [11]:
uk_card_spending_df = get_ONS_dataset(ons_ids[2])
uk_card_spending_df.head()

,v4_1,Data Marking,calendar-years,Time,uk-only,Geography,dd-mm,DayMonth,spend-category,Category
0,99.696013,NaN,2020,2020,K02000001,United Kingdom,17-08,17-08,staple,Staple
1,106.921456,NaN,2022,2022,K02000001,United Kingdom,17-08,17-08,staple,Staple
2,NaN,.,2024,2024,K02000001,United Kingdom,17-08,17-08,work-related,Work Related
3,90.310252,NaN,2020,2020,K02000001,United Kingdom,17-08,17-08,work-related,Work Related
4,86.507714,NaN,2020,2020,K02000001,United Kingdom,17-08,17-08,aggregate,Aggregate


#### 4. Consumer Price Index - inflation rate

In [14]:
cpi_df = get_ONS_dataset(ons_ids[8])
cpi_df.head()

,v4_0,mmm-yy,Time,uk-only,Geography,cpih1dim1aggid,Aggregate
0,128.0,Aug-23,Aug-23,K02000001,United Kingdom,CP0953_0954,"09.5.3/4 Miscellaneous printed matter, station..."
1,112.6,Aug-23,Aug-23,K02000001,United Kingdom,CP052,05.2 Household textiles
2,128.4,Aug-23,Aug-23,K02000001,United Kingdom,CP082_083,08.2/3 Telephone and telefax equipment
3,97.3,Aug-23,Aug-23,K02000001,United Kingdom,CP091,09.1 Audio-visual equipment and related products
4,216.5,Aug-23,Aug-23,K02000001,United Kingdom,CP0733,07.3.3 Passenger transport by air


In [17]:
set(cpi_df['Aggregate'].tolist())

{'01 Food and non-alcoholic beverages',
 '01.1 Food',
 '01.1.1 Bread and cereals',
 '01.1.2 Meat',
 '01.1.3 Fish',
 '01.1.4 Milk, cheese and eggs',
 '01.1.5 Oils and fats',
 '01.1.6 Fruit',
 '01.1.7 Vegetables including potatoes and tubers',
 '01.1.8 Sugar, jam, syrups, chocolate and confectionery',
 '01.1.9 Food products (nec)',
 '01.2 Non-alcoholic beverages',
 '01.2.1 Coffee, tea and cocoa',
 '01.2.2 Mineral waters, soft drinks and juices',
 '02 Alcoholic beverages and tobacco',
 '02.1 Alcoholic beverages',
 '02.1.1 Spirits',
 '02.1.2 Wine',
 '02.1.3 Beer',
 '02.2 Tobacco',
 '03 Clothing and footwear',
 '03.1 Clothing',
 '03.1.2 Garments',
 '03.1.3 Other clothing and clothing accessories',
 '03.1.4 Cleaning, repair and hire of clothing',
 '03.2 Footwear including repairs',
 '04 Housing, water, electricity, gas and other fuels',
 '04.1 Actual rentals for housing',
 '04.2 Owner Occupiers Housing Costs',
 '04.3 Regular maintenance and repair of the dwelling',
 '04.3.1 Materials for mai

In [18]:
cpi_df[cpi_df['Aggregate']=='Overall Index']

,v4_0,mmm-yy,Time,uk-only,Geography,cpih1dim1aggid,Aggregate
7,129.4,Aug-23,Aug-23,K02000001,United Kingdom,CP00,Overall Index
225,104.3,Sep-17,Sep-17,K02000001,United Kingdom,CP00,Overall Index
336,80.0,Jan-06,Jan-06,K02000001,United Kingdom,CP00,Overall Index
453,93.5,Jul-11,Jul-11,K02000001,United Kingdom,CP00,Overall Index
590,47.0,Feb-88,Feb-88,K02000001,United Kingdom,CP00,Overall Index
...,...,...,...,...,...,...,...
52108,81.5,Jun-06,Jun-06,K02000001,United Kingdom,CP00,Overall Index
52257,69.2,Nov-96,Nov-96,K02000001,United Kingdom,CP00,Overall Index
52357,59.9,Aug-91,Aug-91,K02000001,United Kingdom,CP00,Overall Index
52461,91.7,Dec-10,Dec-10,K02000001,United Kingdom,CP00,Overall Index


Need to format as time series and use to calculate inflation rate change --> (past-current/past)x100

#### 5. GDP Monthly Estimates

In [19]:
gdp_df = get_ONS_dataset(ons_ids[6])
gdp_df.head()

,v4_0,mmm-yy,Time,uk-only,Geography,sic-unofficial,UnofficialStandardIndustrialClassification
0,79.3539,Jan-09,Jan-09,K02000001,United Kingdom,F,F : Construction
1,93.8681,Jan-09,Jan-09,K02000001,United Kingdom,B--E,B-E : Production Industries
2,80.5284,Jan-09,Jan-09,K02000001,United Kingdom,G--T,G-T : Index of Services
3,102.3273,Jan-09,Jan-09,K02000001,United Kingdom,A,A : Agriculture
4,82.3913,Jan-09,Jan-09,K02000001,United Kingdom,A--T,A-T : Monthly GDP


In [20]:
gdp_df[gdp_df['sic-unofficial']=='A--T']

,v4_0,mmm-yy,Time,uk-only,Geography,sic-unofficial,UnofficialStandardIndustrialClassification
4,82.3913,Jan-09,Jan-09,K02000001,United Kingdom,A--T,A-T : Monthly GDP
5,102.3331,Nov-22,Nov-22,K02000001,United Kingdom,A--T,A-T : Monthly GDP
13,84.0562,Sep-06,Sep-06,K02000001,United Kingdom,A--T,A-T : Monthly GDP
16,98.5964,Sep-18,Sep-18,K02000001,United Kingdom,A--T,A-T : Monthly GDP
23,66.2490,Dec-97,Dec-97,K02000001,United Kingdom,A--T,A-T : Monthly GDP
...,...,...,...,...,...,...,...
1593,67.8739,Jan-99,Jan-99,K02000001,United Kingdom,A--T,A-T : Monthly GDP
1596,68.6516,Jun-99,Jun-99,K02000001,United Kingdom,A--T,A-T : Monthly GDP
1602,81.2726,Apr-05,Apr-05,K02000001,United Kingdom,A--T,A-T : Monthly GDP
1609,99.3054,Apr-19,Apr-19,K02000001,United Kingdom,A--T,A-T : Monthly GDP


Need to format as time series

### Exploring Bank of England API

#### 1. Mortgage Approval Rates

In [3]:
url = 'https://www.bankofengland.co.uk/boeapps/database/_iadb-fromshowcolumns.asp?' \
    'csv.x=yes&Datefrom=01/Feb/1990&Dateto=now&SeriesCodes=LPMVTVU&CSVF=TT&UsingCodes=Y&VPD=Y&VFD=N'

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/54.0.2840.90 '
                  'Safari/537.36'
}

r = requests.get(url, headers=headers)
mortgages_df = pd.read_csv(io.StringIO(r.text), header=2)
mortgages_df.head()

,DATE,LPMVTVU
0,30 Apr 1993,90963
1,31 May 1993,91684
2,30 Jun 1993,100672
3,31 Jul 1993,93576
4,31 Aug 1993,83478


In [7]:
mortgages_df.tail()

,DATE,LPMVTVU
364,31 Aug 2023,48063
365,30 Sep 2023,44293
366,31 Oct 2023,48927
367,30 Nov 2023,50096
368,31 Dec 2023,36197


#### 2. Interest Rates

In [23]:
import datetime

today = datetime.date.today()
formatted_today = today.strftime("%d/%b/%Y")

# url = f'https://www.bankofengland.co.uk/boeapps/database/_iadb-fromshowcolumns.asp?' \
#     f'csv.x=yes&Datefrom=01/Feb/1990&Dateto={formatted_today}&SeriesCodes=IUMAAMIH&CSVF=TT&UsingCodes=Y&VPD=Y&VFD=N'

url = f'https://www.bankofengland.co.uk/boeapps/database/_iadb-fromshowcolumns.asp?' \
    f'csv.x=yes&Datefrom=01/Feb/1990&Dateto=now&SeriesCodes=IUMABEDR&CSVF=TT&UsingCodes=Y&VPD=Y&VFD=N'

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/54.0.2840.90 '
                  'Safari/537.36'
}

r = requests.get(url, headers=headers)
interest_df = pd.read_csv(io.StringIO(r.text), header=2)
interest_df.head()

,DATE,IUMABEDR
0,28 Feb 1990,14.875
1,31 Mar 1990,14.875
2,30 Apr 1990,14.875
3,31 May 1990,14.875
4,30 Jun 1990,14.875


In [24]:
interest_df.tail()

,DATE,IUMABEDR
403,30 Sep 2023,5.25
404,31 Oct 2023,5.25
405,30 Nov 2023,5.25
406,31 Dec 2023,5.25
407,31 Jan 2024,5.25


### HM Land Registry Data

Need to webscrape urls to make sure updates itself accurately, but will use these just to test importing the data

In [34]:
ppd_url = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv'
ukhpi_url = 'http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/Indices-2023-11.csv?utm_medium=GOV.UK&utm_source=datadownload&utm_campaign=index&utm_term=9.30_17_01_24'
avg_price_url = 'http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/Average-prices-2023-11.csv?utm_medium=GOV.UK&utm_source=datadownload&utm_campaign=average_price&utm_term=9.30_17_01_24'

#### 1. Price Paid Data

File is 4.3gb so potentially would cause performance issues for dashboard. Will avoid using unless absolutely necessary.

In [39]:
#full ppd data is too big to read in at once using pandas
# ppd_df = pd.read_csv(ppd_url)
# ppd_df.head()

# r = requests.get(ppd_url, headers=headers)
# ppd_df = pd.read_csv(io.StringIO(r.text), header=None)
# ppd_df.head()

#### 2. UKHPI Data

In [28]:
ukhpi_df = pd.read_csv(ukhpi_url)
ukhpi_df.head()

,Date,Region_Name,Area_Code,Index
0,1968-04-01,Northern Ireland,N92000001,3.300420
1,1968-04-01,England,E92000001,1.680067
2,1968-04-01,Wales,W92000004,2.119327
3,1968-04-01,Scotland,S92000003,2.108087
4,1968-04-01,London,E12000007,1.096815


In [38]:
set(ukhpi_df['Region_Name'].tolist())

{'Aberdeenshire',
 'Adur',
 'Amber Valley',
 'Angus',
 'Antrim and Newtownabbey',
 'Ards and North Down',
 'Argyll and Bute',
 'Armagh City Banbridge and Craigavon',
 'Arun',
 'Ashfield',
 'Ashford',
 'Babergh',
 'Barking and Dagenham',
 'Barnet',
 'Barnsley',
 'Basildon',
 'Basingstoke and Deane',
 'Bassetlaw',
 'Bath and North East Somerset',
 'Bedford',
 'Belfast',
 'Bexley',
 'Birmingham',
 'Blaby',
 'Blackburn with Darwen',
 'Blackpool',
 'Blaenau Gwent',
 'Bolsover',
 'Bolton',
 'Boston',
 'Bournemouth Christchurch and Poole',
 'Bracknell Forest',
 'Bradford',
 'Braintree',
 'Breckland',
 'Brent',
 'Brentwood',
 'Bridgend',
 'Brighton and Hove',
 'Broadland',
 'Bromley',
 'Bromsgrove',
 'Broxbourne',
 'Broxtowe',
 'Buckinghamshire',
 'Burnley',
 'Bury',
 'Caerphilly',
 'Calderdale',
 'Cambridge',
 'Cambridgeshire',
 'Camden',
 'Cannock Chase',
 'Canterbury',
 'Cardiff',
 'Carmarthenshire',
 'Castle Point',
 'Causeway Coast and Glens',
 'Central Bedfordshire',
 'Ceredigion',
 'Cha

#### 3. Average Price Paid Data

In [35]:
avg_price_df = pd.read_csv(avg_price_url)
avg_price_df.head()

,Date,Region_Name,Area_Code,Average_Price,Monthly_Change,Annual_Change,Average_Price_SA
0,1968-04-01,Northern Ireland,N92000001,3661.485500,0.0,NaN,NaN
1,1968-04-01,England,E92000001,3408.108064,0.0,NaN,NaN
2,1968-04-01,Wales,W92000004,2885.414162,0.0,NaN,NaN
3,1968-04-01,Scotland,S92000003,2844.980688,0.0,NaN,NaN
4,1968-04-01,London,E12000007,4418.489911,0.0,NaN,NaN


In [37]:
set(avg_price_df['Region_Name'].tolist())

{'Aberdeenshire',
 'Adur',
 'Amber Valley',
 'Angus',
 'Antrim and Newtownabbey',
 'Ards and North Down',
 'Argyll and Bute',
 'Armagh City Banbridge and Craigavon',
 'Arun',
 'Ashfield',
 'Ashford',
 'Babergh',
 'Barking and Dagenham',
 'Barnet',
 'Barnsley',
 'Basildon',
 'Basingstoke and Deane',
 'Bassetlaw',
 'Bath and North East Somerset',
 'Bedford',
 'Belfast',
 'Bexley',
 'Birmingham',
 'Blaby',
 'Blackburn with Darwen',
 'Blackpool',
 'Blaenau Gwent',
 'Bolsover',
 'Bolton',
 'Boston',
 'Bournemouth Christchurch and Poole',
 'Bracknell Forest',
 'Bradford',
 'Braintree',
 'Breckland',
 'Brent',
 'Brentwood',
 'Bridgend',
 'Brighton and Hove',
 'Broadland',
 'Bromley',
 'Bromsgrove',
 'Broxbourne',
 'Broxtowe',
 'Buckinghamshire',
 'Burnley',
 'Bury',
 'Caerphilly',
 'Calderdale',
 'Cambridge',
 'Cambridgeshire',
 'Camden',
 'Cannock Chase',
 'Canterbury',
 'Cardiff',
 'Carmarthenshire',
 'Castle Point',
 'Causeway Coast and Glens',
 'Central Bedfordshire',
 'Ceredigion',
 'Cha

### Save all datasets for dashboard building

In [25]:
# cpi_df.to_csv('cpi_data.csv', index=False)
# gdp_df.to_csv('monthly_gdp_data.csv', index=False)
# mortgages_df.to_csv('mortgage_approvals_data.csv', index=False)
interest_df.to_csv('interest_rates_data.csv', index=False)
# ukhpi_df.to_csv('ukhpi_data.csv', index=False)
# avg_price_df.to_csv('avg_price_data.csv', index=False)

Still need to sort out webscraping the csv addresses for HMLR data

### Webscraping URLs for HMLR data

In [42]:
def find_data_url(parent_url, substring):
    """Finds url on a webpage using given substring for matching.
    
    Webscrapes weboage to find all URLs then looks for strings that contain substring.
    If more than one match found will fail to run, substring must be unique to desired data url.
    
    Parameters:
    -----------
    parent_url (str) - URL of webpage to webscrape to find link to data
    sustring (str) - substring to match to identify correct link for data
    
    Returns:
    --------
    data_url (str) - URL of dataset need to download
    """
    # identify all urls
    reqs = requests.get(parent_url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
 
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
        
    # find url for data download
    substr = substring
    results = [i for i in links if substr in i]
    data_url = list(set(results))
    
    if len(data_url) > 1:
        print('Substring did not return unique match. Please provide more suitable substring.')
    else:
        data_url = data_url[0]
        return data_url

In [48]:
parent_url = 'https://www.gov.uk/government/statistical-data-sets/uk-house-price-index-data-downloads-november-2023'
ukhpi_substr = 'UK-HPI-full-file'
avg_price_substr = 'Average-prices-20'

In [49]:
find_data_url(parent_url, ukhpi_substr)

'http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/UK-HPI-full-file-2023-11.csv?utm_medium=GOV.UK&utm_source=datadownload&utm_campaign=full_fil&utm_term=9.30_17_01_24'

In [50]:
find_data_url(parent_url, avg_price_substr)

'http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/Average-prices-2023-11.csv?utm_medium=GOV.UK&utm_source=datadownload&utm_campaign=average_price&utm_term=9.30_17_01_24'